# Regresion usando redes neuronales

In [0]:
# Use seaborn for pairplot
!pip install seaborn

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

Obtenga la data.

Primero se obtiene el dataset.

In [0]:
from google.colab import drive
drive.mount('/content/drive2', force_remount=True)

In [0]:
dataset=pd.read_csv("/content/drive2/My Drive/Colab Notebooks/preprocesamiento/train_v5_sal.csv")
dataset.pop('Unnamed: 0')
dataset = dataset.iloc[:, 0:60]
dataset.head()


Importelo usando pandas.

### Limpie la data

El set de datos contiene algunos valores desconocidos.

In [0]:
dataset.isna().sum()

En caso de encontrar datos faltantes, se sustituyen por 0

In [0]:
dataset = dataset.fillna(0)

### Dividamos la data en entrenamiento y validación

Se divide el  set de datos en un set de entrenamiento y otro para validar.



In [0]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
val_dataset = dataset.drop(train_dataset.index)
print(train_dataset)

### Inspeccione la data

Revise rapidamente la distribucion conjunta de un par de columnas de el set de entrenamiento.

In [0]:
sns.pairplot(train_dataset[["SalePrice", "LotArea", "OverallQual", "YearBuilt"]], diag_kind="kde")

Tambien revise las estadisticas generales:

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("SalePrice")
train_stats = train_stats.transpose()
train_stats

### Separe las caracteristicas de las etiquetas.

Separe el valor objetivo, o la "etiqueta" 
de las caracteristicas. Esta etiqueta es el valor que entrenara el modelo para predecir.

In [0]:
train_labels = train_dataset.pop('SalePrice')
val_labels = val_dataset.pop('SalePrice')

### Normalice la data

Revise otra vez el bloque de `train_stats` que se presento antes y note la diferencia de rangos de cada caracteristica.

Es una buena práctica normalizar funciones que utilizan diferentes escalas y rangos. Aunque el modelo * podría * converger sin normalización de características, dificulta el entrenamiento y hace que el modelo resultante dependa de la elección de las unidades utilizadas en la entrada.

Nota: Aunque generamos intencionalmente estas estadísticas solo del conjunto de datos de entrenamiento, estas estadísticas también se utilizarán para normalizar el conjunto de datos de prueba. Necesitamos hacer eso para proyectar el conjunto de datos de prueba en la misma distribución en la que el modelo ha sido entrenado.

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_val_data = norm(val_dataset)
normed_train_data.head()

Estos datos normalizados es lo que usaremos para entrenar el modelo.

Precaución: las estadísticas utilizadas para normalizar las entradas aquí (media y desviación estándar) deben aplicarse a cualquier otro dato que se alimente al modelo, junto con la codificación de un punto que hicimos anteriormente. Eso incluye el conjunto de pruebas, así como los datos en vivo cuando el modelo se usa en producción.

## El modelo

### Construye el modelo

Construyamos nuestro modelo. Aquí, utilizaremos un modelo `secuencial` con dos capas ocultas densamente conectadas y una capa de salida que devuelve un único valor continuo. Los pasos de construcción del modelo se envuelven en una función, `build_model`, ya que crearemos un segundo modelo, más adelante.

In [0]:
def build_model():
  model = keras.Sequential([
    layers.Dense(100, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(50, activation='relu'),
    layers.Dense(1, activation='relu')
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
  return model

model = build_model()

### Inspeccione el modelo

Use el método `.summary` para imprimir una descripción simple del modelo

In [0]:
model.summary()


Ahora pruebe el modelo. Tome un lote de ejemplos `10` de los datos de entrenamiento y llame a` model.predict` en él.

In [0]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result


Parece estar funcionando, y produce un resultado de la forma y tipo esperados.

### Entrenar a la modelo

Entrene el modelo durante 1000 épocas y registre la precisión de entrenamiento y validación en el objeto `history`.

In [0]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 2000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

Visualice el progreso de entrenamiento del modelo usando las estadísticas almacenadas en el objeto `history`.

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [$]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,100000])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,10000000000])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['rmse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_rmse'],
           label = 'Val Error')
  plt.ylim([0,100000])
  plt.legend()
  plt.show()


plot_history(history)

Este gráfico muestra poca mejora, o incluso degradación en el error de validación después de aproximadamente 100 épocas. Actualicemos la llamada `model.fit` para detener automáticamente el entrenamiento cuando el puntaje de validación no mejore. Utilizaremos una * devolución de llamada de EarlyStopping * que pruebe una condición de entrenamiento para cada época. Si transcurre una cantidad determinada de épocas sin mostrar mejoría, entonces detiene automáticamente el entrenamiento.

Puedes obtener más información sobre esta devolución de llamada [Aca](https://www.tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/EarlyStopping).

In [0]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

Veamos qué tan bien generaliza el modelo al usar el conjunto ** test **, que no usamos al entrenar el modelo. Esto nos dice qué tan bien podemos esperar que el modelo prediga cuándo lo usamos en el mundo real.

In [0]:
loss, mae, mse = model.evaluate(normed_val_data, val_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

### Haga Predicciones

Finalmente, prediga los valores de MPG utilizando datos en el conjunto de pruebas:

In [0]:
val_predictions = model.predict(normed_val_data).flatten()

plt.scatter(val_labels, val_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([0, 600000], [0, 600000])


Parece que nuestro modelo predice razonablemente bien. Echemos un vistazo a la distribución de errores.

In [0]:
error = val_predictions - val_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")

No es del todo gaussiano, pero podríamos esperar eso porque el número de muestras es muy pequeño.

## Reentrenamiento con todos los datos

In [0]:
todo_labels = dataset.pop('SalePrice')


In [0]:
normed_data = norm(dataset)
normed_data.head()

In [0]:
model = build_model()

#early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=50)

history = model.fit(normed_data, todo_labels, epochs=870,
                    validation_split = 0, verbose=0, callbacks=[early_stop, PrintDot()])

#plot_history(history)

## Generar archivo para Kaggle


* 

In [0]:
dataset2=pd.read_csv("/content/drive2/My Drive/Colab Notebooks/preprocesamiento/test_v5_sal.csv")
dataset2.pop('Unnamed: 0')
dataset2 = dataset2.iloc[:, 0:59]
dataset2.head()
print(dataset2)

In [0]:
dataset2.isna().sum()

In [0]:
dataset2 = dataset2.fillna(0)
dataset2.head()

In [0]:

normed_test2_data = norm(dataset2)
normed_test2_data.head()

In [0]:
ynew = model.predict(normed_test2_data)
print(ynew)
dfsal = pd.DataFrame(ynew)
dfsal.columns = ['SalePrice']
print (dfsal)


In [0]:
my_list = list(range(1461,2920))
print(my_list)
dfid = pd.DataFrame(my_list)
dfid.columns = ['Id'] 
print (dfid)
resultados = pd.concat([
    dfid,dfsal
], axis=1)
#resultados.ignore_index()
print(resultados)
resultados.to_csv("/content/drive2/My Drive/Colab Notebooks/preprocesamiento/resultados_v12_sal.csv", index=False)